In [ ]:
!pip install datasets transformers evaluate peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 17.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 M

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [ ]:
# load dataset
dataset = load_dataset('shawhin/imdb-truncated')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

In [ ]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [ ]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/trained_model/model")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/trained_model/tokenizer")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_data = dataset['train']
validation_data = dataset['validation']

In [ ]:
train_data = train_data.map(tokenize_function)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
validation_data = validation_data.map(tokenize_function)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:

# Set the maximum sequence length
max_length = 512

# Initialize lists to store the actual and predicted labels
true_labels = []
predicted_labels = []

# Iterate over the training data to get predictions
for text, label in zip(train_data['text'], train_data['label']):
    # Tokenize the input text and truncate to the maximum length
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=max_length, truncation=True).to("cpu")
    with torch.no_grad():
        logits = model(inputs).logits
    predictions = torch.argmax(logits, dim=1)

    true_labels.append(label)
    predicted_labels.append(predictions.tolist()[0])

# Calculate accuracy
accuracy_score = accuracy.compute(predictions=predicted_labels, references=true_labels)

print(f"Training accuracy: {accuracy_score['accuracy']}")

Training accuracy: 0.914


In [ ]:

# Set the maximum sequence length
max_length = 512

# Initialize lists to store the actual and predicted labels
true_labels = []
predicted_labels = []

# Iterate over the training data to get predictions
for text, label in zip(validation_data['text'], validation_data['label']):
    # Tokenize the input text and truncate to the maximum length
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=max_length, truncation=True).to("cpu")
    with torch.no_grad():
        logits = model(inputs).logits
    predictions = torch.argmax(logits, dim=1)

    true_labels.append(label)
    predicted_labels.append(predictions.tolist()[0])

# Calculate accuracy
accuracy_score = accuracy.compute(predictions=predicted_labels, references=true_labels)

print(f"validation accuracy: {accuracy_score['accuracy']}")

validation accuracy: 0.877
